In [10]:
from pathlib import Path
import pandas as pd
import re

# Carpeta raíz donde están TODAS las ciudades
root = Path("C:/Users/Jose Ruiz/Desktop/booking_output")

dfs = []

for city_dir in root.iterdir():
    if not city_dir.is_dir():
        continue
    ciudad = city_dir.name.replace("-", " ").strip()

    # Busca el CSV dentro de cada carpeta de hotel (con fecha)
    for csv_path in city_dir.glob("*/reviews_newest_first.csv"):
        # ===== limpiar nombre del hotel (carpeta padre) =====
        raw = csv_path.parent.name
        # quita bloque final tipo "YYYY MM DD HH mm ss"
        raw = re.sub(r"\s*\d{4}(?:\s+\d{2}){5}\s*$", "", raw)
        # quita otros números, cambia -/_ por espacios y colapsa espacios
        raw = re.sub(r"\d+", "", raw)
        raw = re.sub(r"[-_]+", " ", raw)
        hotel = re.sub(r"\s+", " ", raw).strip().title()

        # ===== leer CSV =====
        try:
            df = pd.read_csv(csv_path, encoding="utf-8")
        except UnicodeDecodeError:
            df = pd.read_csv(csv_path, encoding="latin-1")

        df["hotel"] = hotel
        df["ciudad"] = ciudad
        dfs.append(df)

# ===== unir y guardar =====
final = pd.concat(dfs, ignore_index=True)
final.to_csv(root / "reviews_booking_venezuela.csv", index=False, encoding="utf-8-sig")
print("OK →", root / "reviews_booking_venezuela.csv", "| filas:", len(final))


OK → C:\Users\Jose Ruiz\Desktop\booking_output\reviews_booking_venezuela.csv | filas: 23137
